In [3]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
pip install pandarallel

  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=61ca7b8e9de11bdd8c618b50518fc4a525730430133324306dc33e708338de00
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel
Note: you may need to restart the kernel to use updated packages.


In [14]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

2025-08-17 16:07:32.128356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755446852.303311      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755446852.353299      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
df=pd.read_csv('/kaggle/input/english-to-marathi-sentence-dataset/data.csv')
df.head()

,Unnamed: 0,English,Marathi
0,0,Go.,जा.
1,1,Go.,जा.
2,2,Go.,जा.
3,3,Go.,जा.
4,4,Hi.,हाय.


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118964 entries, 0 to 118963
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  118964 non-null  int64 
 1   English     118964 non-null  object
 2   Marathi     118964 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [8]:
df = df.drop_duplicates(subset=['English', 'Marathi']).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102959 entries, 0 to 102958
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  102959 non-null  int64 
 1   English     102959 non-null  object
 2   Marathi     102959 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.4+ MB


In [9]:
df.head()

,Unnamed: 0,English,Marathi
0,0,Go.,जा.
1,4,Hi.,हाय.
2,5,Run!,धावा!
3,6,Run.,धावा.
4,7,Who?,WHO?


# english preprocessing

In [10]:
import nltk
import spacy
import re

In [11]:
nlp=spacy.load('en_core_web_lg')

In [12]:
def preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_punct]
    return " ".join(tokens)

# marathi preprocessing

In [15]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

In [16]:
factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("mr")

In [17]:
def marathi_preprocessing(text):
    text = normalizer.normalize(text)  
    text = re.sub(r"[^\u0900-\u097F\s\u0966-\u096F]", "", text) 
    text = re.sub(r"\s+", " ", text).strip() 
    return text

# implementation

In [19]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)  

df['English'] = df['English'].parallel_apply(preprocessing)
df['Marathi'] = df['Marathi'].parallel_apply(marathi_preprocessing)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [20]:
df.head()

,Unnamed: 0,English,Marathi
0,0,go,जा
1,4,hi,हाय
2,5,run,धावा
3,6,run,धावा
4,7,who,


In [21]:
df = df.replace('', np.nan)
df = df.dropna(subset=['Marathi','English'])

In [22]:
df.head()

,Unnamed: 0,English,Marathi
0,0,go,जा
1,4,hi,हाय
2,5,run,धावा
3,6,run,धावा
5,8,fire,आग


In [23]:
df = df.drop_duplicates(subset=['English', 'Marathi']).reset_index(drop=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102702 entries, 0 to 102701
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  102702 non-null  int64 
 1   English     102702 non-null  object
 2   Marathi     102702 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.4+ MB


In [25]:
df.head()

,Unnamed: 0,English,Marathi
0,0,go,जा
1,4,hi,हाय
2,5,run,धावा
3,8,fire,आग
4,11,help,मदत


In [26]:
english_text=df['English'].values
marathi_text_in=df['Marathi'].values

In [69]:
start_token = "<start>"
end_token = "<end>"

marathi_text=[start_token + " " + sent + " " + end_token for sent in marathi_text_in]

In [28]:
english_voc=[text for text in english_text]
marathi_voc=[text for text in marathi_text]

# tokenization english

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
tokenizer_english = Tokenizer(oov_token="<OOV>")

In [31]:
tokenizer_english.fit_on_texts(english_voc)

In [32]:
english_voc_length = len(tokenizer_english.word_index) + 1

In [33]:
sequences_english = tokenizer_english.texts_to_sequences(english_text)

In [34]:
list_english=[len(seq) for seq in sequences_english]
max(list_english),min(list_english)

(47, 1)

In [35]:
max_lenght=max(list_english)
max_lenght

47

In [36]:
english_final = pad_sequences(sequences_english,maxlen=max_lenght, padding='post')

# tokenization marathi

In [37]:
tokenizer_marathi = Tokenizer(oov_token="<OOV>")

In [38]:
tokenizer_marathi.fit_on_texts(marathi_voc)

In [39]:
marathi_voc_length = len(tokenizer_marathi.word_index) + 1

In [76]:
sequences_marathi = tokenizer_marathi.texts_to_sequences(marathi_text)

In [77]:
marathi_voc_length

29347

# input/output creatation

In [178]:
input_enc=[]
input_dec=[]
output_dec=[]

In [179]:
from tqdm import tqdm

In [180]:
for i, j in tqdm(zip(english_final, sequences_marathi)):
    for k in range(1, len(j)):
        input_enc.append(i)         
        input_dec.append(j[:k])      
        output_dec.append(j[k])      

102702it [00:01, 53636.33it/s]


In [181]:
len(input_enc),len(input_dec),len(output_dec)

(681615, 681615, 681615)

In [182]:
marathi_text[:10]

['<start> जा <end>',
 '<start> हाय <end>',
 '<start> धावा <end>',
 '<start> आग <end>',
 '<start> मदत <end>',
 '<start> उडी <end>',
 '<start> थांबा <end>',
 '<start> थांबा <end>',
 '<start> पुढे जा <end>',
 '<start> नमस्कार <end>']

In [183]:
# tokenizer_marathi.word_index

In [184]:
input_dec[:5]

[[2], [2, 380], [2], [2, 4421], [2]]

In [185]:
max_lenght_dec=max([len(seq) for seq in input_dec])
max_lenght_dec

35

In [186]:
input_dec_pad=pad_sequences(input_dec,maxlen=max_lenght_dec, padding='post')

In [187]:
input_dec_pad[:5]

array([[   2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   2,  380,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   2, 4421,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   2,    0,    0,   

In [188]:
input_enc=np.array(input_enc)
input_dec_pad=np.array(input_dec_pad)
output_dec=np.array(output_dec)

In [189]:
input_enc.shape,input_dec_pad.shape,output_dec.shape

((681615, 47), (681615, 35), (681615,))

# model

In [190]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [191]:
latent_dim = 256         # LSTM hidden size
vocab_inp_size = english_voc_length   # adjust based on English vocab size
vocab_tar_size = marathi_voc_length   # adjust based on Marathi vocab size
max_enc_len = input_enc.shape[1]  # 47
max_dec_len = input_dec_pad.shape[1]  # 35

In [192]:
encoder_inputs = Input(shape=(max_enc_len,))
enc_emb = Embedding(vocab_inp_size, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(512, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [193]:
decoder_inputs = Input(shape=(max_dec_len,))
dec_emb_layer = Embedding(vocab_tar_size, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(512, return_sequences=False, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

In [194]:
decoder_dense = Dense(vocab_tar_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [195]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [196]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 47)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 35)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 47, 256)   │  2,459,136 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_5         │ (None, 47)        │          0 │ input_layer_5[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 35, 256)   │  7,512,832 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 512),     │  1,574,912 │ embedding_5[0][0… │
│                     │ (None, 512),      │            │ not_equal_5[0][0] │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ [(None, 512),     │  1,574,912 │ embedding_6[0][0… │
│                     │ (None, 512),      │            │ lstm_5[0][1],     │
│                     │ (None, 512)]      │            │ lstm_5[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 29347)     │ 15,055,011 │ lstm_6[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,176,803 (107.49 MB)

 Trainable params: 28,176,803 (107.49 MB)

 Non-trainable params: 0 (0.00 B)

In [197]:
output_dec = np.expand_dims(output_dec, -1)

In [198]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stop = EarlyStopping(
    monitor='val_loss',    # what metric to monitor
    patience=3,            # stop after 3 epochs of no improvement
    restore_best_weights=True # roll back to best weights
)

In [199]:
history = model.fit([input_enc, input_dec_pad],output_dec,batch_size=128,epochs=20,validation_split=0.2,callbacks=[early_stop])

Epoch 1/20
4261/4261 ━━━━━━━━━━━━━━━━━━━━ 217s 50ms/step - accuracy: 0.1722 - loss: 6.5434 - val_accuracy: 0.1895 - val_loss: 6.3133
Epoch 2/20
4261/4261 ━━━━━━━━━━━━━━━━━━━━ 218s 51ms/step - accuracy: 0.3667 - loss: 4.2410 - val_accuracy: 0.2726 - val_loss: 5.3257
Epoch 3/20
4261/4261 ━━━━━━━━━━━━━━━━━━━━ 218s 51ms/step - accuracy: 0.5179 - loss: 2.7690 - val_accuracy: 0.3237 - val_loss: 4.8597
Epoch 4/20
4261/4261 ━━━━━━━━━━━━━━━━━━━━ 218s 51ms/step - accuracy: 0.6375 - loss: 1.7581 - val_accuracy: 0.3488 - val_loss: 4.7116
Epoch 5/20
4261/4261 ━━━━━━━━━━━━━━━━━━━━ 218s 51ms/step - accuracy: 0.7354 - loss: 1.1027 - val_accuracy: 0.3622 - val_loss: 4.7687
Epoch 6/20
4261/4261 ━━━━━━━━━━━━━━━━━━━━ 218s 51ms/step - accuracy: 0.8117 - loss: 0.7180 - val_accuracy: 0.3633 - val_loss: 4.9609
Epoch 7/20
4261/4261 ━━━━━━━━━━━━━━━━━━━━ 218s 51ms/step - accuracy: 0.8631 - loss: 0.4971 - val_accuracy: 0.3663 - val_loss: 5.1146


In [203]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def translate_sentence(
    input_text,
    tokenizer_english,
    tokenizer_marathi,
    eng_max_len,
    mar_max_len_input,
    preprocess_fn=None,
    start_token="start",
    end_token="end"
):
    if preprocess_fn is not None:
        input_text = preprocess_fn(input_text)

    # Encode input
    enc_seq = tokenizer_english.texts_to_sequences([input_text])
    enc_seq = pad_sequences(enc_seq, maxlen=eng_max_len, padding="post")

    # Get start/end ids
    start_id = tokenizer_marathi.texts_to_sequences([[start_token]])[0][0]
    end_id = tokenizer_marathi.texts_to_sequences([[end_token]])[0][0]

    # Initialize decoder input with <start>
    dec_in = [start_id]

    # Store predicted tokens
    out_words = []

    for _ in range(mar_max_len_input):
        dec_seq = pad_sequences([dec_in], maxlen=mar_max_len_input, padding="post")

        # Predict next token
        preds = model.predict([enc_seq, dec_seq], verbose=0)  # shape = (1, vocab_size)
        pred_id = np.argmax(preds[0])  # take the most probable word

        # Stop if end token
        if pred_id == end_id:
            break

        # Append predicted token
        if pred_id not in (0, start_id, end_id):
            out_words.append(tokenizer_marathi.index_word.get(pred_id, ""))

        # Add to decoder input for next step
        dec_in.append(pred_id)

    return " ".join(out_words)


In [236]:
translate_sentence("i want to talk with you",tokenizer_english,tokenizer_marathi,max_enc_len,max_dec_len,preprocessing)

'मला तुमच्याशी बोलायचे आहे'

In [237]:
import pickle
from tensorflow.keras.models import Model

# Suppose you already have:
# model = your trained model
# tokenizer_english, tokenizer_marathi = your fitted tokenizers

# 1. Save model in .h5 format
model.save("(en-dec)_model.h5")
with open("(en-dec)_model.pkl", "wb") as f:
    pickle.dump(model, f)

# 2. Save tokenizers in pickle format
with open("tokenizer_english.pkl", "wb") as f:
    pickle.dump(tokenizer_english, f)

with open("tokenizer_marathi.pkl", "wb") as f:
    pickle.dump(tokenizer_marathi, f)

print("✅ Model and tokenizers saved successfully!")


✅ Model and tokenizers saved successfully!
